In [4]:
import numpy as np
from cvxopt import matrix, solvers
import pickle

In [16]:
class SoftSVM_QP:
    
    def __init__(self, C=1):
        self.C = C
    
    def fit(self, X, y):
        m, n = X.shape
        P = matrix(((y @ y.T) * (X @ X.T)).astype(np.double)) # half not required
        q = matrix(-np.ones((m, 1)))
        A = matrix(y.T.astype(np.double))
        b = matrix(np.zeros(1))
        G = matrix(np.vstack([-np.eye(m), np.eye(m)]))
        h = matrix(np.hstack([np.zeros(m), self.C*np.ones(m)]))

        self.sol = solvers.qp(P, q, G, h, A, b)
        alphas = np.array(solution['x'])
        sv_idxs = (alphas > 1e-6).flatten()
        self.sv = X[sv_idxs]
        self.sv_y = y[sv_idxs]
        self.alphas = alphas[sv_idxs]
        
        self.b = self.sv_y - np.sum((self.sv @ self.sv.T) * self.alphas * self.sv_y, axis=0)
        self.b = np.mean(self.b)
        
    def predict(self, X):
        prod = np.sum((self.sv @ X.T) * self.alphas * self.sv_y, axis=0) + self.b
        return np.sign(prod)

## Testing

In [11]:
train_data_file = open('../data/part2_data/train_data.pickle', 'rb')
train_data = pickle.load(train_data_file)
train_data_file.close()

s = train_data['data'].shape
train_data['data'] = train_data['data'].reshape(s[0], np.product(s[1:]))

In [13]:
train_data['data']

array([[202, 204, 199, ..., 240, 238, 243],
       [126, 118, 110, ..., 121, 113, 102],
       [251, 249, 250, ..., 105, 121,  26],
       ...,
       [118, 110, 108, ..., 196, 193, 138],
       [ 71,  86,  32, ..., 199, 165, 101],
       [139, 122,  84, ..., 212, 186, 106]], dtype=uint8)

In [17]:
model = SoftSVM_QP()
model.fit(train_data['data'], train_data['labels'])

NameError: name 'C' is not defined

In [ ]:
test_data_file = open('../data/part2_data/test_data.pickle', 'rb')
test_data = pickle.load(test_data_file)
test_data_file.close()

s = test_data['data'].shape
test_data['data'] = test_data['data'].reshape(s[0], np.product(s[1:]))

In [ ]:
preds = model.predict(test_data['data'])

In [ ]:
# check prediction accuracy